In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_log_error

In [ ]:
# 202004 데이터와 예측 데이터 셋
y_raw=pd.read_csv('/content/drive/My Drive/data/dacon_data/202004.csv')
y_pred_raw=pd.read_csv('/content/drive/My Drive/Dacon/submission/07_28/sub_amtonly19_xgb.csv')

In [ ]:
y_raw

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,3,779600,4
1,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,2,3,14,2427700,15
2,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,1,4,18,2184600,18
3,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,2,4,21,3653100,23
4,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,60s,1,5,15,2765500,18
...,...,...,...,...,...,...,...,...,...,...,...,...
1350317,202004,충북,충주시,휴양콘도 운영업,충북,충주시,20s,1,1,5,77000,5
1350318,202004,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,6,92000,6
1350319,202004,충북,충주시,휴양콘도 운영업,충북,충주시,40s,2,3,5,193000,5
1350320,202004,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,5,86000,7


In [ ]:
# 제 예측 파일엔 id 열이 포함되어 있는데 오류났다는 게 이거 때문인 거 같아요.
# 지금은 예외처리 해줘서 id열 없는 셋도 잘 돌아갈 겁니다.
y_pred_raw

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,1.034339e+08
1,1,202004,강원,골프장 운영업,4.057097e+09
2,2,202004,강원,과실 및 채소 소매업,9.916076e+08
3,3,202004,강원,관광 민예품 및 선물용품 소매업,3.002757e+07
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,2.000000e+00
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.577582e+09
1390,1390,202007,충북,한식 음식점업,2.324971e+10
1391,1391,202007,충북,호텔업,3.184740e+07
1392,1392,202007,충북,화장품 및 방향제 소매업,6.143858e+08


In [ ]:
def dacon_rmsle(jj_04_data,pred_data):
  y_pd=jj_04_data[['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']].groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).agg(['sum']).reset_index()
  y_pd.columns=(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT'])

  sido_list = list(set(pred_data['CARD_SIDO_NM']))
  std_list=list(set(pred_data['STD_CLSS_NM']))
  jeju_nan = []
  for sido in sido_list:
    for std in std_list:
      jeju_nan.append([202004,sido,std,0])
  jeju_nan_pd=pd.DataFrame(jeju_nan,columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT'])
  jeju_nc=pd.concat([y_pd,jeju_nan_pd],ignore_index=True)
  jeju_yn=jeju_nc[['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']].groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).agg(['sum']).reset_index()
  
  y_pred04=pred_data[pred_data['REG_YYMM']==202004]
  y_set_n=jeju_yn.merge(y_pred04,on=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'],how='inner')
  if 'id' in y_set_n.columns:
    y_set_n.columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','REG_YYMM_p','CARD_SIDO_NM_p','STD_CLSS_NM_p','real_AMT','id','pred_AMT']
  else:
    y_set_n.columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','REG_YYMM_p','CARD_SIDO_NM_p','STD_CLSS_NM_p','real_AMT','pred_AMT']

  y_set_n['real_AMT']=np.where(y_set_n['real_AMT']<1,0,y_set_n['real_AMT'])
  y_set_n['pred_AMT']=np.where(y_set_n['pred_AMT']<1,0,y_set_n['pred_AMT'])

  y_jj_n=y_set_n[y_set_n['CARD_SIDO_NM']=='제주']
  y_othersn=y_set_n[y_set_n['CARD_SIDO_NM']!='제주']
  
  y_jjn_real=y_jj_n['real_AMT'].values.flatten()
  y_jjn_pred=y_jj_n['pred_AMT'].values.flatten()

  y_othersn_real=y_othersn['real_AMT'].values.flatten()
  y_othersn_pred=y_othersn['pred_AMT'].values.flatten()
  
  jjn_score=np.sqrt(mean_squared_log_error(y_jjn_real,y_jjn_pred))
  othersn_score=np.sqrt(mean_squared_log_error(y_othersn_real, y_othersn_pred))

  return 'RMSLE SCORE : {}'.format((jjn_score*3+othersn_score*16)/19)

In [ ]:
dacon_rmsle(y_raw,y_pred_raw)

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


'RMSLE SCORE : 2.943301846089051'